In [1]:
!pip3 install pyspark

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("importing and data slicing") # name should be quniue for less conflicts

In [5]:
flights = spark.read.csv('./nyc/nycflights.csv', header = True)
airines = spark.read.csv('./nyc/nyc_airlines.csv', header = True)

AttributeError: 'Builder' object has no attribute 'read'

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
flights.createOrReplaceTempView('flights')
airlines.createOrReplaceTempView('airlines')
airports.createOrReplaceTempView('airports')


In [ ]:
query = """ SELECT * FROM flights """
result = spark.sql(query)
result.show()


In [ ]:
# challange 1: which airline has the highest number of delayed departures
query1 = """SELECT a.name,
COUNT(*) as num_delay FROM flights f
LEFT JOIN airlines a ON f.carrier = a.carrier
WHERE f.dep_delay > 0 
GROUP BY a.name
ORDER BY num_delay DESC
LIMIT 1"""

In [ ]:
result1 = spark.sql(query1)
result.show()

In [ ]:
# challange 2: fastest araival in which airport
query2 = """SELECT a.name, 
AVG(arr_delay) as mean_delay 
FROM flights f JOIN aiports on f.dest = a.faa
"""

In [ ]:
# challange 3 : jfk to sea aiport samma avg maximum and minimum airtime

In [ ]:
flights.printSchema()